# Face Recognition – MLOPS_INTERNSHIP_TASK 06!


Task Description 📄

❄️ Create a program that perform below mentioned task upon recognizing a particular face. 

📌 When it recognize your face then - 

👉 It send mail to your mail id by writing this is face of your_name.

👉 Second it send whatsapp message to your friend, it can be anything. 

📌 When it recognize second  face, it can be your friend or family members face.

👉 Create EC2 instance in the AWS using CLI. 

👉 Create 5 GB EBS volume and attach it to the instance. 


## Step 1 - Create Training Data

In [1]:
# Import necessary libraries
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import threading
import subprocess
import time

# Loading HAAR cascase classifier
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Function for detecting faces
def face_extract(img):
    faces = classifier.detectMultiScale(img, 1.3, 5)
    
    if faces == ():
        return None
    
    # Croppping the face
    for (x, y, width, height) in faces:
        crop_face = img[y:y+height+10, x:x+width+10]
        
    return crop_face

# Initialize Webcam
user_id = input('ENTER USER-NUMBER : ')
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:
    ret, frame = cap.read()
    if face_extract(frame) is None:
        print("Face not found")
        pass
    
    else:
        count +=1
        face = cv2.resize(face_extract(frame), (200,200))
        
        # Save file in specified directory with unique name
        file_name_path = './faces/user/user_' + str(user_id) + "_" + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        face1 = cv2.flip(face,1)
        cv2.putText(face1, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Crop face', face1)
        
    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

ENTER USER-NUMBER : 1


<ipython-input-1-cef4befef2f0>:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():


In [2]:
# Import necessary libraries
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import threading

# Loading HAAR cascase classifier
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Function for detecting faces
def face_extract(img):
    faces = classifier.detectMultiScale(img, 1.3, 5)
    
    if faces == ():
        return None
    
    # Croppping the face
    for (x, y, width, height) in faces:
        crop_face = img[y:y+height+10, x:x+width+10]
        
    return crop_face

# Initialize Webcam
user_id = input('ENTER USER-NUMBER : ')
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:
    ret, frame = cap.read()
    if face_extract(frame) is None:
        print("Face not found")
        pass
    
    else:
        count +=1
        face = cv2.resize(face_extract(frame), (200,200))
        
        # Save file in specified directory with unique name
        file_name_path = './faces/user2/user_' + str(user_id) + "_" + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        face1 = cv2.flip(face,1)
        cv2.putText(face1, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Crop face', face1)
        
    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

ENTER USER-NUMBER : 2


<ipython-input-2-ae76b75f569c>:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():


## Step 2 - Train Model

In [3]:
# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    user_id=int(os.path.split(image_path)[-1].split("_")[1])
    Labels.append(user_id)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)
model  = cv2.face_LBPHFaceRecognizer.create()

# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))

#saving our created model in a file
model.save('ModelFiles/model.yml')
print("Model trained sucessefully")

Model trained sucessefully


In [4]:
# Get the training data we previously made
data_path = './faces/user2/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    user_id=int(os.path.split(image_path)[-1].split("_")[1])
    Labels.append(user_id)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)
model  = cv2.face_LBPHFaceRecognizer.create()

# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))

#saving our created model in a file
model.save('ModelFiles/model2.yml')
print("Model 2 trained sucessefully")

Model 2 trained sucessefully


## Step 3 - Run Our Facial Recognition

In [5]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

user_id=0
name = ""

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)
model1 = cv2.face_LBPHFaceRecognizer.create();
model1.read('ModelFiles/model.yml')

while True:
    
    ret, frame = cap.read()
    image1, face1 = face_detector(frame)
    cv2.putText(image1, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
    cv2.imshow('Face Recognition', image1 )
    
    try:
        face1 = cv2.cvtColor(face1, cv2.COLOR_BGR2GRAY)
        results1 = model1.predict(face1)
        user_id1 = results1[0]
        score1 = results1[1]

        if score1 < 500:
            confidence1 = int( 100 * (1 - (score1)/400) )
            if confidence1 > 85:
                cv2.imshow('Face Recognition', image1 )
                display_string = str(confidence1) + '% Confident it is bhargava'
                cv2.putText(image1,display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,255), 2)
                print("Bhargava.... check your email and also a whatsapp message is sent")
                %run ./sending_message.ipynb
                break           
        else:
            print("Unknown user found")
            
    except:
        cv2.putText(image1, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)

        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()

<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-5-1924320c1152>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Bhargava.... check your email and also a whatsapp message is sent
13
04
In 14 seconds web.whatsapp.com will open and after 20 seconds message will be delivered


In [ ]:
cap.release()

In [6]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

user_id=0
name = ""

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

# Open Webcam
cap = cv2.VideoCapture(0)
model2 = cv2.face_LBPHFaceRecognizer.create();
model2.read('ModelFiles/model2.yml')

while True:
    
    ret, frame = cap.read() 
    image2, face2 = face_detector(frame)
    cv2.putText(image2, "looking for right face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
    cv2.imshow('Face Recognition', image2 )
    
    try:
        face2 = cv2.cvtColor(face2, cv2.COLOR_BGR2GRAY)
        results2 = model2.predict(face2)
        user_id2 = results2[0]
        score2 = results2[1]

        if score2 < 500:
            confidence2= int( 100 * (1 - (score2)/400) )
            if confidence2 > 85:
                cv2.imshow('Face Recognition', image2 )
                display_string = str(confidence2) + '% Confident it is shahid'
                cv2.putText(image2, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,255), 2)
                print("You are shahid so Check your AWS")
                os.system("aws ec2 create-key-pair  --key-name mlopskey")
                a = os.system("aws ec2 run-instances  --image-id ami-0d8d212151031f51c  --count 1  --instance-type t2.micro --key-name mlopskey  --subnet-id subnet-42f6810e   --tag-specifications  ResourceType=instance,Tags=[{Key=Name,Value=mlopstask6}]")
                if a == 0:
                    print("\t\t\t Success : Instance Launched")
                b = os.system("aws ec2 create-volume --volume-type gp2 --size 5 --availability-zone us-east-2c  --tag-specifications  ResourceType=volume,Tags=[{Key=Name,Value=mlopstask6}]")
                if b == 0:
                    print("\t\t\t Success : Volume Created")

                time.sleep(30)
                ins_id = subprocess.getoutput("aws ec2 describe-instances --filters Name=tag:Name,Values=mlopstask6 --query Reservations[*].Instances[*].[InstanceId] --output text")
                vol_id = subprocess.getoutput("aws ec2 describe-volumes  --filters Name=tag:Name,Values=mlopstask6* --query Volumes[*].[VolumeId] --output text")
                c = os.system("aws ec2 attach-volume --instance-id {} --volume-id {} --device /dev/sdf".format(ins_id,vol_id))
                if c == 0:
                    print("\t\t\t Success : Volume ATTACHED")
                break
            
        else:
            print("Unknown user found")
            cv2.imshow('Face Recognition', image2 )
            display_string = str(confidence2) + '% Confident it is not shahid'
            cv2.putText(image2, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,0), 2)
            
    except:
        cv2.putText(image2, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()

<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-6-e7f74144bfad>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


You are shahid so Check your AWS
			 Success : Instance Launched
			 Success : Volume Created
			 Success : Volume ATTACHED


task completed